# <span style='color:Blue'> Used Car Price Prediction  </span>

## Problem Statement: Pandemic-Driven Purchase

In the recent days, due to pandemic, many individuals are looking forward to owning a vehicle who otherwise preferred public transport.
The fear of contacting virus while using a public transport, prompted consumers to owning ga vehicle. Considering affrordability we can observe a trend in growth of used car market.
Second hand cars are preferred mostly by those who cannot afford to buy new cars at higher prices.
This is a attempt to predict used car price with help of data so that customers can be offered competitive prices. 

### Please vote up and share your feedback in the comment box, if you like this notebook. 

<div style="width:100%;text-align: center;"> <img src="https://ichef.bbci.co.uk/news/976/cpsprodpb/10316/production/_121162366_gettyimages-1233138884.jpg" width="500px"/> </div>

### Data Description
Selling_Price : The price of the used car in INR Lakhs.<br>
Name : The brand and model of the car.<br>
Location : The location in which the car is being sold or is available for purchase.<br>
Year : The year or edition of the model.<br>
Kilometers Driven : The total kilometers driven in the car by the previous owner(s) in KM.<br>
Fuel Type : The type of fuel used by the car. (Petrol, Diesel, Electric, CNG, LPG)<br>
Transmission : The type of transmission used by the car. (Automatic / Manual)<br>
Seller_Type : Whether the seller an Individual, Dealer or Trustmark Dealer.<br>
Owner Type : Whether the ownership is Firsthand, Second hand or other.<br>
Mileage : The standard mileage offered by the car company in kmpl(Petrol/Diesel) or km/kg(CNG/ LPG)<br>
Current_Mileage : Current mileage claimed by the seller.<br>
Engine : The displacement volume of the engine in CC.<br>
Power : The maximum power of the engine in bhp<br>
Seats : The number of seats in the car.<br>
New_Price : Latest price of vehicle.<br>

### Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from wordcloud import WordCloud
from tabulate import tabulate
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

ModuleNotFoundError: No module named 'matplotlib.pyplot'

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
%matplotlib inline

class color:
   BLUE = '\033[94m'
   BOLD = '\033[1m'
   END = '\033[0m'

In [ ]:
train_data = pd.read_csv('train_data_2.csv')
test_data = pd.read_csv('test_data_2.csv')

# <span style='color:Blue'> Data Overview  </span>

Let us look at the data.

### Train Data

In [ ]:
train_data.columns

In [ ]:
train_data.head()

In [ ]:
print(color.BOLD + "There are {} rows and {} columns in the dataset.".format(train_data.shape[0],train_data.shape[1]),"\n"+ color.END)
print("The first column is unnamed which seems to be the index which can be deleted and reset the index.","\n")
train_data.drop('Unnamed: 0', axis=1, inplace=True)
print(color.BOLD +color.BLUE +"Let's look at the data types available in the dataset"+ color.END)
train_data.info()
print(color.BOLD +color.BLUE +"\n","Summary statistics of dataset"+ color.END)
train_data.describe()

### Selling Price
selling price is given in INR Lakhs, we will multiply the columns with 1,00,000. to get the correct price.

In [ ]:
train_data['Selling_Price'] = train_data['Selling_Price'].apply(lambda x: x*100000)

### Test Data

In [ ]:
test_data.head()

In [ ]:
print(color.BOLD +"There are {} rows and {} columns in the dataset.".format(test_data.shape[0],test_data.shape[1]),"\n"+ color.END)
print("The first column is unnamed which seems to be the index which can be deleted and reset the index.","\n")
test_data.drop('Unnamed: 0', axis=1, inplace=True)
print(color.BOLD +color.BLUE +"Let's look at the data types available in the dataset"+ color.END)
test_data.info()
print(color.BOLD +color.BLUE +"\n","Summary statistics of dataset"+ color.END)
test_data.describe()

There are columns such as Current_Mileage, Engine, Power, Seats and New_Price which have null-values.<br>
Also, data type of appropriate columns have to be changed to meet the requirement.

## <span style='color:Maroon'> Converting to appropriate data type  </span>


Though data types of Year and Seats are 'int64' and 'float64' respectively, using them with the same datatype will not be useful for our evaluation.<br>
we need to convert these parameters into datetime and categorical(nominal) respectively.<br>
Also, Location, Fuel_type, Transmission, Owner_Type, Seller_Type can all be changed to categorical variables.<br>

### Seats Column
There are records where Seats is given as 0 which is misleading. This can be assumed that number of seats is not recorded for those enteries. We will consider this as NaN an impute them with make and model information.

In [ ]:
def seats_(df):
    df['Seats'].replace(0, np.nan, inplace=True)
    df['Seats'] = df['Seats'].astype('category')

In [ ]:
seats_(train_data)
seats_(test_data)

### Year column
we will calculate the age of vehicle from Year column and drop Year column.

In [ ]:
current_year = datetime.datetime.now().year #get current year
def veh_age(df):
    df["vehicle_age"] = df['Year'].apply(lambda x: current_year-x)  # substract to get the year delta
    df.drop('Year', axis=1, inplace=True)

In [ ]:
veh_age(train_data)
veh_age(test_data)

### Current_Mileage and Mileage
Current_Mileage and Mileage have units which have to be removed and these should be converted into appropriate datatype 'float64'.
When Fuel_Type is CNG/ LPG units for Current_Mileage and mileage are km/kg, when Fuel_Type is Petrol/ Diesel units are kmpl.
In addition there are values with 'null' in the column, null can be converted to NaN and imputed later on.
Units can be removed using regex or str operations.

In [ ]:
pattern = '\s\D+[/]*\D+'

def mileage_(df):
    df['Current_Mileage'] = df['Current_Mileage'].replace(to_replace = pattern, value = '', regex = True)
    df['Current_Mileage'] = df['Current_Mileage'].astype(float)
    df['Mileage'] = df['Mileage'].replace(to_replace = pattern, value = '', regex = True)
    df['Mileage'] = df['Mileage'].astype(float)

In [ ]:
mileage_(train_data)
mileage_(test_data)

# <span style='color:Blue'> Feature Engineering  </span>

## <span style='color:Maroon'> Missing  Value Treatment  </span>

### Check for null/ nan in Dataset

In [ ]:
def checknull(df):
    # printing column name where null is present
    col_name = df.isnull().sum(axis=0).sort_values(ascending = False)
    print(col_name)    

In [ ]:
print(color.BOLD +"printing column name where null is present in train data"+ color.END, '\n')
checknull(train_data)
print(color.BOLD +"printing column name where null is present in test data"+ color.END, '\n')
checknull(test_data)

In [ ]:
print(color.BOLD +"There are many null values in New_Price. It is very difficult to impute those values. Hence we will drop that column from both test and train data.","\n"+ color.END)

In [ ]:
train_data.drop('New_Price', axis=1, inplace=True)

In [ ]:
test_data.drop('New_Price', axis=1, inplace=True)

In [ ]:
print(color.BOLD +'Rows with NaN in Mileage in train data',"\n"+ color.END)
train_data[train_data['Mileage'].isna()]

In [ ]:
print(color.BOLD +"There are {} row entries for electric vehicle in train data.".format(train_data['Fuel_Type'][train_data['Fuel_Type'] == 'Electric'].count()),"\n"+ color.END)

In [ ]:
print(color.BOLD +"There are {} row entries for electric vehicle in test data.".format(test_data['Fuel_Type'][test_data['Fuel_Type'] == 'Electric'].count()),"\n"+ color.END)

For electric vehicle instead of mileage there is a parameter called range.<br>Range is Distance km/Charge.
Also there is no Electric in fuel type in test data. To maintain uniformity these two rows can be dropped.

In [ ]:
# Get indexes where Fuel_Type column has value Electric
indexNames = train_data[train_data['Fuel_Type'] == 'Electric'].index
 
# Delete these row indexes from dataFrame
train_data.drop(indexNames , inplace=True, axis=0)

There are records where Current_Mileage is given as 0 which is misleading. This can be assumed that the Current_Mileage is not recorded.
We will consider this as NaN an replace during imputation with the mileage value given by the manufacturer.<br>

## <span style='color:Maroon'> Removing suffixes from values in a column  </span>


For Engine units are CC.<br>For Power units are bhp. In addition there are values with 'null bhp' in the column, null can be converted to NaN and imputed later on. <br>
Units can be removed using regex or str operations. 

In [ ]:
pattern = '\s\D+[/]*\D+'

def curmil_eng_pow(df):
    df['Current_Mileage'].replace(0, np.nan, inplace=True)
    df['Mileage'].replace(0, np.nan, inplace=True)
    print("There are {} null in Current_Mileage.Replace them with the mileage value".format(df['Current_Mileage'].isna().sum()),"\n")
    df.Current_Mileage.fillna(df.Mileage, inplace=True)
    
    df['Engine'] = df['Engine'].replace(to_replace = pattern, value = '', regex = True)
    df['Engine'] = df['Engine'].astype("float").astype("Int64")
    df['Power'] = pd.to_numeric(df['Power'].str.lower().str.split().str.get(0).str.replace('null',''), errors='coerce')

In [ ]:
print(color.BOLD +"Treating train data"+ color.END, '\n')
curmil_eng_pow(train_data)
print(color.BOLD +"Treating test data"+ color.END, '\n')
curmil_eng_pow(test_data)

print("Remove units and change datatype for Engine and Power")


## <span style='color:Maroon'> Splitting columns  </span>

We can extract the Make and model infomation from name.<br>
We can split Make and Model into separate columns which can be used further  and drop name column.

In [ ]:
def get_make_and_model(df):
    
    #Get the Make from the name
    make_list=list(df['Name'].str.lower())
    i=0

    for item in make_list:
        make_list[i] = item.split(' ')[0]

        i=i+1

    # replace formated Names into Make  
    df['Make']=make_list
    
    #Get the Model from the name
    model_list=list(df['Name'].str.lower())
    i=0

    for item in model_list:
        model_list[i] = item.split(' ')[0]+'_'+item.split(' ')[1]

        i=i+1

    # replace formated Names into Model  
    df['Model']=model_list
    
    #Drop name column
    df.drop('Name', axis=1, inplace=True)

In [ ]:
get_make_and_model(train_data)
get_make_and_model(test_data)

## <span style='color:Maroon'> Impute missing values  </span>

Impute missing values in 'Power','Mileage', 'Seats','Engine','Seller_Type'.<br>

Vehicles with different variants of under same Make and Model tend to have same Engine capacity and Power.<br>
All missing values in Engine and Power column, missing values can be imputed based on the Make and Model information available.<br>

In [ ]:
train_data=  train_data.sort_values(['Make','Model'])
for column in ['Power','Mileage', 'Seats','Engine','Seller_Type']:
    train_data[column].fillna(method='ffill', inplace=True)

In [ ]:
test_data=  test_data.sort_values(['Make','Model'])
for column in ['Power','Mileage', 'Seats','Engine','Seller_Type']:
    test_data[column].fillna(method='ffill', inplace=True)

## <span style='color:Maroon'> Checking Distribution of Numeric Variables  </span>


In [ ]:
numeric_traindf = train_data.select_dtypes(include=['int64','float']).drop(columns=['Selling_Price'])

row_nums = 2  # how many rows of plots
col_nums = 3  # how many plots per row

# Create the subplots
fig, axes = plt.subplots(nrows=row_nums, ncols=col_nums, figsize=(15, 10))

for i, column in enumerate(numeric_traindf.columns):
    sns.distplot(numeric_traindf[column],ax=axes[i//col_nums,i%col_nums])
    #sns.histplot(numeric_traindf[column],ax=axes[i//col_nums,i%col_nums], color="red", kde=True)


    
# "i//ncols gives the floor division which is the row when you are working left to right then top to bottom.
# i%ncols will give you the integer remainder which is the column when you are working left to right top to bottom."

* Distribution of kilometers_driven is highly positive skewed
* Distribution of Engine, Power and vehicle_age are moderately positive skewed. 

In [ ]:
numeric_traindf.agg(['skew', 'kurtosis']).transpose()

As rule of thumb, skewness can be interpreted like this: <br>

Skewness<br>
Fairly Symmetrical	-0.5 to 0.5<br>
Moderate Skewed	-0.5 to -1.0 and 0.5 to 1.0<br>
Highly Skewed	< -1.0 and > 1.0<br>

## <span style='color:Maroon'> Correlation between variables  </span>

### Relation between Mileage, Engine and Power

Over period, values specified by manufaturer such as Mileage, Engine and Power will eventually change due to wear and tear, hence we will drop these columns.<br>
Also, the values are given by the manufacturer cannot be a deciding factor to buy a vehicle.<br>
For understanding we can check the correlation with the target value(used car price).<br>
These are the data given by the manufaturer.<br>

In [ ]:
%matplotlib inline
# Step 0 - Read the dataset, calculate column correlations and make a seaborn heatmap
df_cor = train_data.select_dtypes(include=['int64','float']).drop(columns=['Kilometers_Driven', 'vehicle_age'])
corr = df_cor.corr()
print(corr)

fig, ax = plt.subplots(figsize=(10,10))
#ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap= sns.diverging_palette(20, 220, as_cmap=True), square=True, annot=True, fmt='.1f')
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap= 'Blues', square=True, annot=True, fmt='.2f')

ax.set_xticklabels(ax.get_xticklabels(), rotation=45,horizontalalignment='right');

In [ ]:
sns.pairplot(df_cor)

* selling_Price(used car price) has a moderate correlation with Power and low correlation with the mileage.<br>
* Engine and Power have high correlation with each other.<br>
* Mileage and Current_Mileage have high correlation with each other.<br>
* Based on correlation we will Drop 'Mileage','Engine' columns.

In [ ]:
drop_column_list=['Mileage','Engine','Power']
train_data.drop(drop_column_list, axis=1, inplace=True)
train_data.reset_index(drop=True, inplace=True)

In [ ]:
test_data.drop(drop_column_list, axis=1, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## <span style='color:Maroon'> Outlier Treatment  </span>

In [ ]:
train_data.select_dtypes(include=['int64','float']).describe()

### Relation between Kilometers_Driven and vehicle_age

Third Quartile and above values in Kilometers_Driven have very high values. We will look at them closely to get an understanding about them.

In [ ]:
# Get indexes where Fuel_Type column has value Electric
train_data[train_data['Kilometers_Driven'] >= 300000]

There is one entry where Kilometers_Driven is 6500000 which is not likely possible considering age of the vehicle.<br> we will replace this value by max of km driven based on age of that vehicle.<br>

In [ ]:
index_=train_data.loc[train_data['Kilometers_Driven'] == train_data['Kilometers_Driven'].max()].index.item()
vehicle_age_=train_data['vehicle_age'][train_data['Kilometers_Driven'] == train_data['Kilometers_Driven'].max()].item()

print(color.BOLD +"Maximum in Kilometers_Driven is {}, this is an outlier".format(train_data['Kilometers_Driven'].max()), "\n"+ color.END)
print(color.BOLD +"Index of this row is {} and age of this vehicle is {}".format(index_,vehicle_age_), "\n"+ color.END)

In [ ]:
train_data['Kilometers_Driven'] = train_data['Kilometers_Driven'].replace(train_data['Kilometers_Driven'].max(),np.NaN)
replace_=train_data['Kilometers_Driven'].groupby(train_data['vehicle_age']).get_group(vehicle_age_).max()
print(color.BOLD +"Maximum Kilometers_Driven by vehicles with age of {} is {}".format(vehicle_age_,replace_), "\n"+ color.END)

In [ ]:
train_data['Kilometers_Driven'].fillna(replace_,inplace=True)

There are very few vehicles which have recorded very high values in Kilometers_Driven, we will limit these values to 300000.

In [ ]:
train_data['Kilometers_Driven'].mask(train_data['Kilometers_Driven'] > 300000, 300000, inplace=True)

In [ ]:
# Set up the axes with gridspec
x=train_data['vehicle_age']
y=train_data['Kilometers_Driven']

# Set up the axes with gridspec
fig = plt.figure(figsize=(8, 8))
grid = plt.GridSpec(4, 4, hspace=0.2, wspace=0.2)
main_ax = fig.add_subplot(grid[:-1, 1:])
y_hist = fig.add_subplot(grid[:-1, 0], xticklabels=[], sharey=main_ax)
x_hist = fig.add_subplot(grid[-1, 1:], yticklabels=[], sharex=main_ax)

# scatter points on the main axes
main_ax.plot(x, y, 'ok', markersize=4, alpha=0.2)

# histogram on the attached axes
x_hist.hist(x, 40, histtype='stepfilled', orientation='vertical', color='skyblue')
#x_hist.invert_yaxis()

y_hist.hist(y, 60, histtype='stepfilled', orientation='horizontal', color='skyblue')
y_hist.invert_xaxis()

# Title 
plt.suptitle('Relation between Kilometers_Driven and vehicle_age', size = 15);

Though we have addressed the skewness in Kilometers_Driven data, from the above plot we can see that distribution of data is skewed.<br>
We will use box-cox transformation to address the skewness and normalize the data.

## <span style='color:Maroon'> Box-Cox Transformation for numeric variables variables  </span>


In [ ]:
# Check if the data type of all columns is same in train _data and test_data
#train_data.info()
#test_data.info()
test_data['Kilometers_Driven'] = test_data['Kilometers_Driven'].astype(float)

In [ ]:
numeric_traindf = train_data.select_dtypes(include=['int64','float']).drop(columns=['Selling_Price']).apply(lambda x: stats.boxcox(x)[0])
numeric_testdf = test_data.select_dtypes(include=['int64','float']).apply(lambda x: stats.boxcox(x)[0])


In [ ]:
# Replace original data with box-cox ransformed data
train_data.loc[:, ['Kilometers_Driven', 'Current_Mileage', 'vehicle_age']] = numeric_traindf[['Kilometers_Driven', 'Current_Mileage', 'vehicle_age']]
test_data.loc[:, ['Kilometers_Driven', 'Current_Mileage', 'vehicle_age']] = numeric_testdf[['Kilometers_Driven', 'Current_Mileage', 'vehicle_age']]


In [ ]:
print(train_data.isnull().values.any())
print(test_data.isnull().values.any())

print(color.BOLD +"There are no more null values in train_data and test data"+ color.END)

We have successfully imputed all the missing values, normalized the data. This way we can be assured that we have not lost any data.
Since the number of missing values is not high we can also choose to delete those rows. But that will lead to loss of data.

## <span style='color:Maroon'> Data Grouping</span>

We can group categories within column for category variables. In this method we will check the unique categories within the categorical variables and reduce the number of categories keeping highly appropriate categories.

In [ ]:
%matplotlib inline
cat_column_list={'Transmission': 'Manual transmission is most popular.',
                 'Fuel_Type':'Many vehicles that are sold are diesel vehicles, followed by petrol vehicles.',
                 'Seller_Type':'Most sellers are indivisual sellers.',
                 'Owner_Type':'Many vehicles that are sold are first hand vehicles.',
                 'Seats':'5 seater vehicles are owned by majority of the users.',
                 'Location':'Popular locaitions where the vehicle is available for purchase are Mumbai, Hyderabad and kochi'}

for column in cat_column_list.keys():
    #uniques = train_data[column].values
    #total_unique=len(list(np.unique(uniques)))
    count_uniques = pd.DataFrame(train_data[column].value_counts()).rename(columns={column:'Total_Count'}).sort_values('Total_Count',ascending=False)
    
    # parameters in format function.
    print(color.BOLD +"Number of unique values in {} is {}".format(column, count_uniques.shape[0]), "\n"+ color.END)
    #print("Unique value count in {}".format(column))
    #print(count_uniques)

    # Create Figure
    fig, ax = plt.subplots(figsize=(5,5))

    ax = sns.barplot(x=count_uniques.index.values.tolist()  , y="Total_Count", data=count_uniques, palette= 'viridis')
    # rotates labels and aligns them horizontally to left 
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=90, ha="left" )

    plt.tight_layout()
    plt.show()
    print("{}".format(cat_column_list[column]))

    print("\n",'-------------------------------------------------------------------------------------------------')

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(14, 9))
count_uniques = pd.DataFrame(train_data['Model'].value_counts()).reset_index().rename(columns={'index':'options','Model':'Total_Count'}).sort_values('Total_Count', ascending=False)

print(color.BOLD +"Number of unique values in Model is {}".format(count_uniques.shape[0]), "\n"+ color.END)

dictionary = pd.Series(count_uniques.Total_Count.values,index=count_uniques.options).to_dict()

wordcloud = WordCloud(max_font_size=800, background_color='white', colormap='viridis', width=500, height=300, max_words=15).generate_from_frequencies(dictionary)

plt.imshow(wordcloud, interpolation='bilinear') # image show
plt.axis('off'); # to off the axis of x and y


In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(14, 9))
count_uniques = pd.DataFrame(train_data['Make'].value_counts()).reset_index().rename(columns={'index':'options','Make':'Total_Count'}).sort_values('Total_Count', ascending=False)

print(color.BOLD +"Number of unique values in Make is {}".format(count_uniques.shape[0]), "\n"+ color.END)

dictionary = pd.Series(count_uniques.Total_Count.values,index=count_uniques.options).to_dict()

wordcloud = WordCloud(max_font_size=800, background_color='white', colormap='viridis', width=500, height=300, max_words=15).generate_from_frequencies(dictionary)

plt.imshow(wordcloud, interpolation='bilinear') # image show
plt.axis('off'); # to off the axis of x and y



In [ ]:
print(color.BOLD +"Aggregate statistics for Selling_price"+ color.END)
train_data['Selling_Price'].describe()

In [ ]:
#train_data[train_data.Selling_Price==train_data.Selling_Price.min()]

vehicle_ = train_data['Model'][train_data.Selling_Price==train_data.Selling_Price.min()]
print("One {} vehicle was sold at {} INR, lowest in the dataset".format(vehicle_.item(), train_data.Selling_Price.min()))
vehicle_ = train_data['Model'][train_data.Selling_Price==train_data.Selling_Price.max()]
print("One {} vehicle was sold at {} INR, highest in the dataset".format(vehicle_.item(), train_data.Selling_Price.max()))

In [ ]:
#train_data['Selling_Price'].loc[train_data['Make'] == 'nissan'].describe()

In [ ]:
sns.set_style("ticks",{'axes.grid' : True})
ax = sns.catplot(data=train_data, x='Selling_Price', y='Make',height=10, color='skyblue')  

# rotates labels and aligns them horizontally to left 
plt.suptitle('Selling Price vs Make of Vehicle', size = 15);

ax.set(xticks=(2e+05,5e+05,1e+06,2e+06,3e+06,4e+06,5e+06,1e+07))
ax.set_xticklabels((2e+05,5e+05,1e+06,2e+06,3e+06,4e+06,5e+06,1e+07), rotation=90)

plt.tight_layout()
plt.show()

print('-------------------------------------------------------------------------------------------------')

* Selling price of maruti vehicles are between 45000 and 1150000 INR.<br>
* Selling price of hyundai vehicles is mostly between 45000 and 1800000 INR but there are some vehicle which were sold for more than 2000000 INR.<br>
* Majority of honda vehicles are sold between 90000 and 1200000 INR.<br>
* Minimum selling price of nissan vehicles is 175000 and maximum selling price is 892000 INR.<br>
* Selling price of maruti vehicles are between 45000 and 1150000 INR.

In [ ]:
sns.set_style("ticks",{'axes.grid' : True})

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(20, 20))

# Plot the orbital period with horizontal boxes
sns.boxplot(data=train_data, x='Selling_Price', y='Make',
            whis=[0, 100], width=.6, color='skyblue')

# Add in points to show each observation
sns.stripplot(data=train_data, x='Selling_Price', y='Make',
              size=4, color=".3", linewidth=0)

# Tweak the visual presentation
ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)

* Selling price of Audi, Mercedes-Benz and BMW vehicles are spread over a wide range.
* The median value of Audi, Mercedes-Benz, BMW and Mini brands of vehicles is above 20Lakhs INR.
* Median value of Jaguar and Land Rover vehicles is above 30Lakh INR.
* 1st Quartile value of Porsche is above 40Lakhs INR.

In [ ]:
count_uniques = pd.DataFrame(train_data['Make'].value_counts()).rename_axis('Make').rename(columns={'Make':'Total_Count'})[24:30]
print(tabulate(count_uniques, headers = ["Make", "Total_Count "], tablefmt="pretty"),'\n')
print("There are very few vehicles in our dataset of {}".format(count_uniques.index.tolist()))
vehicle_ = train_data.loc[train_data['Make'] == 'bentley']
print("There is only one {} vehicle was sold at {} INR".format(vehicle_.Model.item(), vehicle_.Selling_Price.item()))
vehicle_ = train_data.loc[train_data['Make'] == 'ambassador']
print("There is only one {} vehicle was sold at {} INR".format(vehicle_.Model.item(), vehicle_.Selling_Price.item()))
vehicle_ = train_data.loc[train_data['Make'] == 'smart']
print("There is only one {} vehicle was sold at {} INR".format(vehicle_.Model.item(), vehicle_.Selling_Price.item()))

## <span style='color:Maroon'> categorical variables into Binary variables and adding new columns </span>

* If vehicle is in top 15 in Make, encode the column value as 1 else 0 in Make_15_BIN
* If vehicle is in top 15 in Model, encode the column value as 1 else 0 in Model_15_BIN
* If Location is in list of metero cities ['Chennai','Delhi','Mumbai','Kolkata','Ahmedabad','Bangalore','Hyderabad','Pune'], encode the column value as 1 else 0 in Model_15_BIN
* If Fuel_Type is other than Petrol or Diesel, change it to 'Gas_fuel'
* If Owner_Type is other than First or Second, change it to 'Third&above'
* If Seats are other than 2,4,5, change to '6nabove'
* If Seller_Type other than 'Individual' change to 'Dealer'

In [ ]:
# Change the categorical variables into Binary variable

fuel_list= ['Diesel','Petrol']
Owner_Type= ['First','Second']
Seats= [2,4,5]
Seller_Type= ['Individual']
Make_top15_list= train_data.Make.value_counts().index[0:15].to_list()
Model_top15_list= train_data.Model.value_counts().index[0:15].to_list()
Metro_city_list= ['Chennai','Delhi','Mumbai','Kolkata','Ahmedabad','Bangalore','Hyderabad','Pune']

def Binary_variable(df):
    df['Fuel_Type'] = df['Fuel_Type'].apply(lambda x: x if x in fuel_list else 'Gas_fuel')
    df['Owner_Type'] = df['Owner_Type'].apply(lambda x: x if x in Owner_Type else 'Third&above')
    df['Seats'] = df['Seats'].apply(lambda x: x if x in Seats else '6nabove')
    df['Seller_Type'] = df['Seller_Type'].apply(lambda x: x if x in Seller_Type else 'Dealer')
    df['Make_15_BIN'] = df['Make'].apply(lambda x: 1 if x in Make_top15_list else 0)#.astype('category')
    df['Model_15_BIN'] = df['Model'].apply(lambda x: 1 if x in Model_top15_list else 0)#.astype('category')
    df['Location'] = df['Location'].apply(lambda x: 1 if x in Metro_city_list else 0)


Binary_variable(train_data)
Binary_variable(test_data)

Delete Make and Model columns 

In [ ]:
drop_column_list=['Make', 'Model']
train_data.drop(drop_column_list, axis=1, inplace=True)
test_data.drop(drop_column_list, axis=1, inplace=True)
#train_data.info()
#test_data.info()

## <span style='color:Maroon'> Dummy Variable Creation </span>

In [ ]:
# generate binary values using get_dummies
train_df = pd.get_dummies(train_data)
test_df = pd.get_dummies(test_data)
#train_df.shape
#test_df.shape

## <span style='color:Maroon'> Correlation matrix </span>

In [ ]:
corrmat=train_df.corr()
top_corr_features=corrmat.index
plt.figure(figsize=(25,25))
#plot heat map
g=sns.heatmap(train_df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

We can delete columns which have high correlation with other columns.<br>Fuel_Type_Diesel-Fuel_Type_Petrol,<br> Transmission_Automatic-Transmission_Manual,<br> Seller_Type_Dealer-Seller_Type_Individual, <br>Owner_Type_Second-Owner_Type_First,<br> Seats_6nabove-Seats5.0 <br> have high correlation with each other. <br>Hence we can drop
'Fuel_Type_Diesel', 'Transmission_Automatic','Seller_Type_Dealer', 'Owner_Type_Second', 'Seats_6nabove'

In [ ]:
drop_column_list=['Fuel_Type_Diesel', 'Transmission_Automatic','Seller_Type_Dealer', 'Owner_Type_Second', 'Seats_6nabove']

train_df.drop(columns=[col for col in train_df if col in drop_column_list], inplace=True)
test_df.drop(columns=[col for col in test_df if col in drop_column_list], inplace=True)

#train_df.drop(drop_column_list, axis=1, inplace=True)
#test_df.drop(drop_column_list, axis=1, inplace=True)

In [ ]:
#train_df.columns
#test_df.columns
#train_df[train_df.isin([np.nan, np.inf, -np.inf]).any(1)]
#test_df[test_df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
#train_df.to_csv("train_df_check.csv")

## <span style='color:Maroon'> Feature Importance </span>

With feature importance we can understand which features are very important for price prediction 

In [ ]:
X = train_df.iloc[:,1:]  #independent columns
y = train_df.iloc[:,0]    #target column Selling_Price

plt.figure(figsize=(7,7))

model = ExtraTreesRegressor()
model.fit(X,y)

print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers

#plot graph of feature importances for better visualization
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
feature_importances.nlargest(15).plot(kind='barh')
plt.show()

* Transmission type is the very important feature followed by vehicle age
* Current Mileage and Kilometers Driven are almost equally important features

In [ ]:
print("The following are the top 15 features(columns) in the order of decreasing importance that govern the selling pricec of the vehicle.",'\n')
print(feature_importances.nlargest(15).index.tolist())

# <span style='color:Blue'> Model Building  </span>


In [ ]:
train_df.columns

In [ ]:
X=train_df.drop(columns='Selling_Price', axis=1)
y=train_df['Selling_Price']

In [ ]:
# Test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## <span style='color:Maroon'> Random Forest Regressor </span>

We will build a random forest regressor model.<br>
We will also use RandomizedSearchCV.<br> 
* In RandomizedSearchCV not all hyperparameter values are tried out. Instead, a fixed number of hyperparameter settings is sampled from specified probability distributions.

In [ ]:
rfreg=RandomForestRegressor()

In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rfreg, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=123, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)

The result of a hyperparameter optimization is a single set of well-performing hyperparameters that you can use to configure your model.

In [ ]:
print("The best parameters finalized for random forest regressor")
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
## <span style='color:Maroon'> Make predictions on holdout sample </span>

In [ ]:
y_predicted=rf_random.predict(X_test)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_predicted)) 
print('Mean Absolute Percentage Error:',mean_absolute_percentage_error(y_test,y_predicted))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_predicted))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_predicted)))
print('R2 score: ', r2_score(y_test, y_predicted))

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

plt.scatter(y_test, y_predicted)
plt.xlabel('Actual Price')
plt.ylabel("Predicted Price")
plt.title("Actual vs Predicted Price")
ax.set(xticks=(2e+05,5e+05,1e+06,2e+06,3e+06,4e+06,5e+06,1e+07))
ax.set_xticklabels((2e+05,5e+05,1e+06,2e+06,3e+06,4e+06,5e+06,1e+07), rotation=90)
ax.set(yticks=(2e+05,5e+05,1e+06,2e+06,3e+06,4e+06,5e+06,1e+07))
ax.set_yticklabels((2e+05,5e+05,1e+06,2e+06,3e+06,4e+06,5e+06,1e+07))

plt.show()

In [ ]:
sns.distplot(y_test-y_predicted)

# <span style='color:Blue'> Make Predictions on Test Data </span>


In [ ]:
y_predicted=rf_random.predict(test_df)

In [ ]:
final_result=pd.DataFrame(y_predicted, index =list(test_df.index)).rename(columns={0:'Predicted_Selling_Price'})
final_result